In [ ]:
%pip install py-cpuinfo
%pip install gputil

In [ ]:
import psutil
import platform
import cpuinfo
import GPUtil
import subprocess

def get_ram_info():
    ram_gb = round(psutil.virtual_memory().total / (1024 ** 3))
    return f"{ram_gb} GB"

def get_disk_info():
    disk = psutil.disk_usage('/')
    disk_gb = round(disk.total / (1024 ** 3))
    return f"{disk_gb} GB SSD"

def get_cpu_info():
    info = cpuinfo.get_cpu_info()
    cpu_freq = psutil.cpu_freq()
    max_clock = round(cpu_freq.max, 2)
    min_clock = round(cpu_freq.min, 2)
    try:
        lscpu_output = subprocess.check_output("lscpu", shell=True).decode()
        pci_lanes = next((line.split(":")[1].strip() for line in lscpu_output.splitlines() if "PCI" in line), "N/A")
        cores_per_socket = next((line.split(":")[1].strip() for line in lscpu_output.splitlines() if "Core(s) per socket" in line), "N/A")
        threads_per_core = next((line.split(":")[1].strip() for line in lscpu_output.splitlines() if "Thread(s) per core" in line), "N/A")
    except:
        pci_lanes = "N/A"
        cores_per_socket = psutil.cpu_count(logical=False) // psutil.cpu_count()
        threads_per_core = psutil.cpu_count() // psutil.cpu_count(logical=False)

    return f"""
\\textit{{Name}}: {info['brand_raw']} \\\\
\\textit{{Architecture}}: {platform.machine()} \\\\
\\textit{{CPU(s)}}: {psutil.cpu_count()} \\\\
\\textit{{Thread(s) per core}}: {threads_per_core} \\\\
\\textit{{Core(s) per socket}}: {cores_per_socket} \\\\
\\textit{{Max Clock Speed}}: {max_clock} MHz \\\\
\\textit{{Min Clock Speed}}: {min_clock} MHz \\\\
\\textit{{PCI Express Lanes}}: {pci_lanes} \\\\
\\textit{{Integrated Graphics}}: None \\\\
\\textit{{TDP (Thermal Design Power)}}: 140 W \\\\
"""

def get_cache_info():
    try:
        lscpu_output = subprocess.check_output("lscpu -e", shell=True).decode()
        l1d = "192 KiB (6 instances)"
        l1i = "192 KiB (6 instances)"
        l2 = "6 MiB (6 instances)"
        l3 = "8.3 MiB (1 instance)"
    except:
        l1d = l1i = l2 = l3 = "N/A"
    return f"""
\\textit{{L1d cache}}: {l1d} \\\\
\\textit{{L1i cache}}: {l1i} \\\\
\\textit{{L2 cache}}: {l2} \\\\
\\textit{{L3 cache}}: {l3} \\\\
"""

def get_gpu_info():
    gpus = GPUtil.getGPUs()
    if not gpus:
        return "\\textit{Name}: None \\\\"
    gpu = gpus[0]
    return f"""
\\textit{{Name}}: {gpu.name} \\\\
\\textit{{Memory}}: {gpu.memoryTotal} GB HBM2 \\\\
\\textit{{Memory Interface Width}}: 3072 bits \\\\
\\textit{{Memory Bandwidth}}: 652.8 GB/s \\\\
\\textit{{Base Clock Speed}}: 1200 MHz \\\\
\\textit{{Boost Clock Speed}}: 1455 MHz \\\\
\\textit{{Memory Clock Speed}}: 850 MHz (1700 MHz effective) \\\\
\\textit{{CUDA Cores}}: 5120 \\\\
\\textit{{Tensor Cores}}: 640 \\\\
\\textit{{Thermal Design Power (TDP)}}: 250 W \\\\
"""

def generate_latex_table():
    latex = f"""
\\begin{{table*}}[h]
\\centering
\\caption{{Local server configuration}}
\\label{{table:DescriptionOfFeatures}}
{{
\\begin{{tabular}}{{|ll|}}
\\hline
\\textbf{{Component}} & \\textbf{{Configuration}} \\\\
\\hline
Primary Memory (RAM) & {get_ram_info()} \\\\
\\hline
Secondary Memory (Hard Disk) & {get_disk_info()} \\\\
\\hline
Processor & 
\\begin{{tabular}}{{@{{}}l@{{}}}}
{get_cpu_info()}
\\end{{tabular}} \\\\
\\hline
Cache & 
\\begin{{tabular}}{{@{{}}l@{{}}}}
{get_cache_info()}
\\end{{tabular}} \\\\
\\hline
GPU & 
\\begin{{tabular}}{{@{{}}l@{{}}}}
{get_gpu_info()}
\\end{{tabular}} \\\\
\\hline
\\end{{tabular}}
}}
\\end{{table*}}
"""
    return latex

if __name__ == "__main__":
    print(generate_latex_table())
